# Настройка ноутбука

In [2]:
import pandas as pd
import numpy as np
from sklearn import datasets
import datetime as dt

from pandasql import sqldf

In [93]:
# Расширить рабочее поле ноутбука на весь экран
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Ресурсы

[Medium: Query Pandas DataFrame with SQL](https://towardsdatascience.com/query-pandas-dataframe-with-sql-2bb7a509793d#:~:text=Pandasql%20is%20a%20python%20library,the%20SQLite%20table%20using%20SQL)

# Задачи

### Задача Яндекс

```terminal

Имеющиеся данные:

orders - таблица с корпоративными заказами Доставки

SELECT *
FROM orders
LIMIT 3

+----------+------------------------+----------------+-----------+
| order_id | utc_order_created_dttm | corp_client_id | city_name |
+----------+------------------------+----------------+-----------+
| jt5t78h5 | 2022-01-03 12:44:45    | v74hr7b5       | moscow    |
+----------+------------------------+----------------+-----------+
| mxe84nur | 2022-02-01 08:12:57    | zl94g5j6       | moscow    |
+----------+------------------------+----------------+-----------+
| lv84h6w1 | 2022-02-04 22:40:00    | qrp67myn       | spb       |
+----------+------------------------+----------------+-----------+

Нужно вывести топ-3% (не меньше) клиентов по общему числу доставок
за последний месяц в каждом городе
```

#### Решение на собеседовании:

```sql
-- DESC:
-- order_id

-- RESULT:
-- city_name, corp_client_id, cnt_orders
-- top3
-- last month


-- MY SOLUTION:
-- group by city_name, corp_client_id + count(distinct order_id),
-- 

with as 

cnt_table_1 as (
    select
          city_name, corp_client_id
        , count(distinct order_id) as cnt_orders
        , rank_percent() over (partition by city_name order by count(distinct order_id) desc) as pct
    from orders 
    where utc_order_created_dttm >= '2022-04-01'
    group by city_name, corp_client_id
),

cnt_table as (
    select
          city_name, corp_client_id
        , count(distinct order_id) as cnt_orders
    from orders 
    where utc_order_created_dttm >= '2022-04-01'
    group by city_name, corp_client_id
),

top3pct as (
    select
        city_name, corp_client_id, cnt_orders
        , (rank() over (partition city_name order by cnt_orders desc) / count(corp_client_id) over (partition by city_name)) as pct_rank
    from cnt_table
)

select 
    city_name, corp_client_id, cnt_orders 
from top3pct
where pct_rank <= 0.03

-- city1, cl3, 30, 30/60, 1 1/4
-- city1, cl5, 30, 20/60, 1 1/4
-- city1, cl2, 20, 20/60, 2 2/4 50
-- city1, cl1, 10, 10/60, 3 3/4

Чтобы функция в подобной реализации работала правильно в выборке должно быть не меньше 33 клиентов
1/x <= 0.03
1 <= 0.03x
100/3 <= 100x


-- city1, cl3, 30, 30/60, 1 1/4
-- city1, cl5, 30, 20/60, 1 1/4
-- city1, cl2, 20, 20/60, 3 3/4
-- city1, cl1, 10, 10/60, 4 4/4


-- 1 / cnt_users
-- 2 / 
-- 3 
-- ...
-- 100
```

#### Похожая задача:

- [Stratascratch: Most Checkins - Medium](https://platform.stratascratch.com/coding/10053-most-checkins/solutions?code_type=1)

#### Решение ChatGPT

```sql
WITH last_month_orders AS (
    SELECT client_id, city_name, COUNT(*) AS total_orders
    FROM orders
    WHERE date >= current_date - INTERVAL '1 month'
    GROUP BY client_id, city_name
),
ranked_orders AS (
    SELECT client_id, city_name, total_orders,
           ROW_NUMBER() OVER (PARTITION BY city_name ORDER BY total_orders DESC) AS rn,
           COUNT(*) OVER (PARTITION BY city_name) AS total_clients
    FROM last_month_orders
)
SELECT client_id, city_name, total_orders
FROM ranked_orders
WHERE rn <= 0.03 * total_clients;
```

В этом примере мы сначала создаем общий набор данных last_month_orders, в котором подсчитываем общее число заказов для каждого клиента в каждом городе за последний месяц. Затем мы используем аналитическую функцию ROW_NUMBER() для присвоения ранга каждому клиенту в каждом городе на основе числа заказов. Мы также используем аналитическую функцию COUNT(*) для подсчета общего числа клиентов в каждом городе.

В конечном запросе мы выбираем только те строки, у которых ранг находится в пределах топ-3% (0.03) от общего числа клиентов в каждом городе. Это позволяет нам получить топ-3% клиентов по общему числу заказов за последний месяц в каждом городе.

### Задача VK - 1
Есть табличка some_table вида some_id Integer, в которой хранится числовая последовательность от 1 до n с пропусками. Нужно найти начало и конец пропуска данных, если его длина больше k Пример: some_id 1, 2, 3, 9, 10, 11, 12, 20, 23 k = 2 => 4, 8 13, 19.

In [8]:
df1 = pd.DataFrame([1, 2, 3, 9, 10, 11, 12, 20, 23], columns=['some_id'])
df1

,some_id
0,1
1,2
2,3
3,9
4,10
5,11
6,12
7,20
8,23


1. Как выделить пропуски в данных? Ответ: val - lag(val) > 1
2. Как из множества пропусков выделить те, что больше k? Ответ: val - lag(val, -1) > k
3. Как из определить начало и конец пропуска (следующее и предыдущее число)? Ответ: +1, -1
4. В каком виде лучше представить результат (перечисление в одной колонке / распределить по двум колонкам)? Ответ: попробуем оба 

#### Решение sql

Вариант №1 - перечисление в одной колонке

In [91]:
k = 2

query = f"""
with 

begins as (
    select some_id + 1 as val
    from (select some_id, lead(some_id, 1) over () - some_id as diff from df1)
    where diff > {k} + 2
),

ends as (
    select some_id - 1 as val
    from (select some_id, some_id - lag(some_id, 1) over () as diff from df1)
    where diff > {k} + 2
)

select val from ends
union
select val from begins
"""

pysqldf = lambda q: sqldf(q, globals())
pysqldf(query)

,val
0,4
1,8
2,13
3,19


Вариант №2 - начало и конец в различных колонках

In [85]:
k = 2

query = f"""
with diff_df as (
    select 
            some_id +1 as begin
          , lead(some_id) over () - 1 as end
          , lead(some_id) over () - some_id as diff
    from df1
)
select begin, end from diff_df where diff > {k} + 2
"""

pysqldf = lambda q: sqldf(q, globals())
pysqldf(query)

,begin,end
0,4,8
1,13,19


#### Решение pandas

In [37]:
df = df1.copy()

df['lag'] = df['some_id'].shift(1)
df['diff'] = df['some_id'] - df['lag']

df

,some_id,lag,diff
0,1,NaN,NaN
1,2,1.0,1.0
2,3,2.0,1.0
3,9,3.0,6.0
4,10,9.0,1.0
5,11,10.0,1.0
6,12,11.0,1.0
7,20,12.0,8.0
8,23,20.0,3.0


### Задача VK - 2

Дана таблица:

 --- 
dt (DateTime)       | currency (String) | quote (float) |
 --- | --- | --- | 
2020-05-31 11:00:01 |      EUR/USD      | 1.1455 |
2020-05-31 11:00:03 |      USD/RUB      | 70.05 |
2020-05-31 11:00:04 |      USD/RUB      | 70.01 |


Вывести  начальный (самый ранний) курс для каждой пары валют 

In [85]:
df = pd.DataFrame({
    'dt': [dt.datetime(2022,5,31, 11,0,1),dt.datetime(2022,5,31, 11,0,2),dt.datetime(2022,5,31, 12,0,1),dt.datetime(2022,5,31, 12,0,2),dt.datetime(2022,5,31, 12,0,3),],
    'currency': ['EUR/USD','EUR/USD', 'USD/RUB', 'USD/RUB', 'USD/RUB'],
    'quote': [1.1455, 1.15, 70.05, 70.01, 70.06]
})

df

,dt,currency,quote
0,2022-05-31 11:00:01,EUR/USD,1.1455
1,2022-05-31 11:00:02,EUR/USD,1.1500
2,2022-05-31 12:00:01,USD/RUB,70.0500
3,2022-05-31 12:00:02,USD/RUB,70.0100
4,2022-05-31 12:00:03,USD/RUB,70.0600


In [86]:
# solution №1: cte
query = """
with min_dts as (
select
      min(dt) as min_dt
    , currency
from
    df
group by currency

)

select 
      t.currency
    , t.quote
from
    df t inner join min_dts m on 
        t.dt = m.min_dt
        and
        t.currency = m.currency
"""

# solution №2: sub-query
sql = """

select dt, currency, quote
from df
where dt in (select min(dt) from df group by currency)
    

"""

pysqldf = lambda q: sqldf(q, globals())
pysqldf(query)

,currency,quote
0,EUR/USD,1.1455
1,USD/RUB,70.0500


### Задачи VK - 3

```sql
-- BoxOffice
-- id, dt, country, film_id, value

-- Films
-- id, film_name, creation_date, genre ...

-- FilmsRating
-- id, film_id, rating (самая высокая оценка, если есть)


-- DESC:
-- BoxOffice: строка - показ фильма + собранные деньги
-- Films: строка - фильм
-- FilmsRating: строка - фильм с рейтингом, если найден

-- RESULT:
-- Вывести страну в которой шло наибольшее количество фильмов 1 января 2020
-- country

-- where dt = 01.01.2020
-- group by country count(film_id)

with
country_rank as (
    select
          country
        -- , count(film_id) as films_cnt
        -- , rank() over (order by count(film_id) desc) as cnt_rank
        count(film_id)
    from BoxOffice
    where dt = '01.01.2020'
    grouby country
    order by cnt_rank desc
)
select country from country_rank limit 1


-- Вывести названия топ 5 самых кассовых фильмов за 2019 год
-- film_name

with
top5 as (
    select
          film_id
        , sum(value) as sum_val
    from BoxOffice
    where to_char(dt, 'YYYY') = '2019'
    group by film_id
    order sum_val desc
    limit 5
)
select 
    f.film_name
from top5 t left join Films f on t.film_id = f.id

-- Посчитать кол-во фильмов шедших в прокате в апреле 2023
-- , которым еще не выставлен рейтинг

select 
not_rated as (
    (select id from BoxOffice where to_char(dt, 'MM.YYYY') = '04.2023')
    difference
    (select id from FilmsRating)
)
select count(id) from not_rated


select 
    count(distinct film_id) as film_cnt 
from BoxOffice 
where 
        to_char(dt, 'MM.YYYY') = '04.2023' 
    and film_id not in (select film_id from FilmsRating)



-- Возникает ошибка Нехватки памяти. Почему так?

select 
	uniqExact(user_id) as users 
from 
	table 
where 
	(timestamp between ‘2020-01-01 00:00:00’ and ‘2020-01-01 23:59:59’)
	and 
	(type = 1 or type is null)
	
-- Чтение журналов - портал
-- Пользователи заполняют анкету: пол, возраст, ... 100 пользователей
-- Мы формируем выдачу статей
-- Дневная аудитория 10 000 000 пользователей
-- Система монетизации:
-- 1) Реклама между статьями
-- 2) Подписка - премиальный контент

-- Цветовые раскладки для кнопок.

-- Как разбить аудиторию на 5 групп?
-- Поделить на 5 частей

-- Ошибку какого рода позволяет нам минимизировать T-тест?
-- Какую ошибку нам позволяет минимизировать MDE?

-- Показать красную кнопку Если ..., показать зеленую Если ....
-- 1) Пользователи должны быть независимы
-- 2) Одинаково распределенные признаки
-- 3) Одинаковый размер групп

-- Мешок с фигурками: 
-- кубики, шарики, конусы, 
-- разные цвета
-- матовые, блеклые

-- Как составить 5 репрезентативных выборок большого мешка?
-- Закон больших чисел?

-- 1) случайно брать 
-- 2) стратификация + стратифицирование среднее
-- 3) кластерную - пытаюсь найти совокупности, найти ортоганальные группу и уже по ним извлекать.

-- Какие задачи?
-- Увеличить рост в условиях, когда он перестает расти сам по себе.
-- Большая работа про расзметке, проверке событий.

-- Сколько всего аналитиков?
-- Сейчас 6 человек.
-- Продуктовый аналитик непосредственно взаимодействует с продуктовым менеджерам.
```

### Задачи Tinkoff

In [41]:
# История количества и сумм транзакций клиентов с января 2021 по декабрь
history_client_trans = pd.DataFrame({
    # месяц транзакции
    'time_key': [
        dt.date(2021,1,1), dt.date(2021,1,2), dt.date(2021,1,3), dt.date(2021,1,4), dt.date(2021,1,5), dt.date(2021,1,6), 
        dt.date(2021,2,1), dt.date(2021,2,2), dt.date(2021,2,3), dt.date(2021,2,4), dt.date(2021,2,5), dt.date(2021,2,6),
    ], 
    # уникальный идентификатор клиента
    'client_rk': [
        1001232545, 1001232546, 1001232547, 1001232548, 1001232549, 1001232550,
        1001232565, 1001232566, 1001232567, 1001232568, 1001232569, 1001232570,
    ],
    # id банка, в котором обслуживаются клиент
    'bank_id': [
        'M123', 'M123', 'M456', 'M456', 'M123', 'M123',
        'M123', 'M123', 'M456', 'M456', 'M456', 'M456',
    ],
    # код филиала регистрации клиента
    'market_key': [
        'VN', 'VN', 'VN', 'N', 'N', 'N',
        'N', 'VN', 'VN', 'VN', 'VN', 'VN',
    ],
    # сумма транзакций, руб
    'trans_amt': [
        25000, 20000, 15000, 10000, 30000, 10000,
        25000, 30000, 35000, 40000, 20000, 40000,
    ],
    # количество транзакций, шт
    'trans_cnt': [
        25, 20, 15, 10, 30, 10,
        25, 30, 35, 40, 20, 40,
    ],
})

# Справочник названия банков
bank_info = pd.DataFrame({
    # id банка, в котором обслуживается клиент
    'bank_id': ['M123', 'M456'],
    # название банка
    'bank_name': ['TINKOFF', 'SBER'],
})

# Справочник филиалов
market_info = pd.DataFrame({
    # код филиала регистрации клиента
    'market_key': ['VN', 'N'],
    # название филиала
    'market_name': ['Великий Новгород', 'Новосибирск'],
})

In [42]:
history_client_trans

,time_key,client_rk,bank_id,market_key,trans_amt,trans_cnt
0,2021-01-01,1001232545,M123,VN,25000,25
1,2021-01-02,1001232546,M123,VN,20000,20
2,2021-01-03,1001232547,M456,VN,15000,15
3,2021-01-04,1001232548,M456,N,10000,10
4,2021-01-05,1001232549,M123,N,30000,30
5,2021-01-06,1001232550,M123,N,10000,10
6,2021-02-01,1001232565,M123,N,25000,25
7,2021-02-02,1001232566,M123,VN,30000,30
8,2021-02-03,1001232567,M456,VN,35000,35
9,2021-02-04,1001232568,M456,VN,40000,40


In [43]:
bank_info

,bank_id,bank_name
0,M123,TINKOFF
1,M456,SBER


In [44]:
market_info

,market_key,market_name
0,VN,Великий Новгород
1,N,Новосибирск


1.	Вывести помесячно среднее, максимальное и минимальное количество и сумму транзакций по всем клиентам

In [45]:
query = """
select 
      strftime('%m', time_key)
    , avg(trans_amt)
    , max(trans_amt)
    , min(trans_amt)
    , sum(trans_amt)
    
    , avg(trans_cnt)
    , max(trans_cnt)
    , min(trans_cnt)
    , sum(trans_cnt)

from
    history_client_trans
group by
    strftime('%m', time_key)
"""

pysqldf = lambda q: sqldf(q, globals())
pysqldf(query)

,"strftime('%m', time_key)",avg(trans_amt),max(trans_amt),min(trans_amt),sum(trans_amt),avg(trans_cnt),max(trans_cnt),min(trans_cnt),sum(trans_cnt)
0,01,18333.333333,30000,10000,110000,18.333333,30,10,110
1,02,31666.666667,40000,20000,190000,31.666667,40,20,190


2.	Вывести распределение клиентов в январе 2021г по филиалам и банкам (в абсолютных значениях)

In [55]:
query = """
select 
      market_name
    , bank_name
    , count(t.client_rk) as n_clients
from
    history_client_trans t 
        left join bank_info b on t.bank_id = b.bank_id
        left join market_info m on t.market_key = m.market_key
where
    strftime('%m', t.time_key) = '01'
group by
    market_name, bank_name
"""

pysqldf = lambda q: sqldf(q, globals())
pysqldf(query)

,market_name,bank_name,n_clients
0,Великий Новгород,SBER,1
1,Великий Новгород,TINKOFF,2
2,Новосибирск,SBER,1
3,Новосибирск,TINKOFF,2


3.	Вывести распределение клиентов в январе 2021г по филиалам и банкам (в %, сумма должна равняться 100%)

In [67]:
query = """
select 
      market_name
    , bank_name
    , count(t.client_rk) as n_client
    , count(t.client_rk)*100/(count(*) over ()) as percent
from
    history_client_trans t 
        left join bank_info b on t.bank_id = b.bank_id
        left join market_info m on t.market_key = m.market_key
where
    strftime('%m', t.time_key) = '01'
group by
    market_name, bank_name
"""

pysqldf = lambda q: sqldf(q, globals())
pysqldf(query)

,market_name,bank_name,n_client,percent
0,Великий Новгород,SBER,1,25
1,Великий Новгород,TINKOFF,2,50
2,Новосибирск,SBER,1,25
3,Новосибирск,TINKOFF,2,50


4.	В каждом филиале по каждому банку вывести топ-5 клиентов по количеству транзакций в феврале 2021 (2021-02)

In [83]:
# TODO
query = """
select 
      market_name
    , bank_name
    , client_rk 
    , trans_cnt
    , sum(trans_cnt) over (partition by market_name, bank_name) as trans_sum
from
    history_client_trans t 
        left join bank_info b on t.bank_id = b.bank_id
        left join market_info m on t.market_key = m.market_key
where
    strftime('%m', t.time_key) = '02'
"""

pysqldf = lambda q: sqldf(q, globals())
pysqldf(query)

,market_name,bank_name,client_rk,trans_cnt,trans_sum
0,Великий Новгород,SBER,1001232567,35,135
1,Великий Новгород,SBER,1001232568,40,135
2,Великий Новгород,SBER,1001232569,20,135
3,Великий Новгород,SBER,1001232570,40,135
4,Великий Новгород,TINKOFF,1001232566,30,30
5,Новосибирск,TINKOFF,1001232565,25,25


5.	Вывести всех клиентов, которые НЕ совершали транзакции в TINKOFF в феврале 2021 (2021-02-01)

In [ ]:
query = """
select 

from
    
"""

pysqldf = lambda q: sqldf(q, globals())
pysqldf(query)